## This Colab File contains the code for Calculating HRV Parameters from Raw IBI Onmnyk data

***Mounting drive in order to use files stored on google drive***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Setting the path to appropriate directory.***(This is the place where we are saving the HRV Parameters ,calculated from Raw IBI values)

In [ ]:
#Results Folder contain two folders- HRV_Parameters & Predictions_Stress_Analysis
cd /content/drive/MyDrive/FYP/first/final/Results/HRV_Parameters

/content/drive/MyDrive/FYP/first/final/Results/HRV_Parameters


***Importing neccessary modules***

In [ ]:
import os
import pandas as pd
import numpy as  np

**Code for calculation of HRV Parameters**

In [ ]:
def time_domain_pm(nn_intervals):
  nn_absdiff=np.diff(nn_intervals)
  nn_sqabs_diff= np.power(nn_absdiff,2)
  HR = 60000/(np.mean(nn_intervals))
  MEAN_RR = np.mean(nn_intervals)
  MEDIAN_RR = np.median(nn_intervals)
  SDRR=np.std(nn_intervals)
  SDSD=np.std(nn_absdiff)
  RMSSD= np.sqrt(np.mean(nn_sqabs_diff))
  NN25=nn_absdiff[np.where(nn_absdiff > 25)]
  NN50= nn_absdiff[np.where(nn_absdiff > 50)]
  try:
    pNN25 = float(len(NN25))/float(len(nn_absdiff))
  except:
    pNN25 = np.nan
  try:
    pNN50 = float(len(NN50))/float(len(nn_absdiff))
  except:
    pNN50 = np.nan

  return [MEAN_RR, MEDIAN_RR, SDRR, RMSSD, SDSD, HR, pNN25, pNN50]

Generating HRV Parameters for Raw IBI data

In [ ]:
for i in os.listdir('/content/drive/MyDrive/FYP/first/final/Data/Omnyk_data/Testing_data_of_ten_patients'): #Path of folder that cotains testing data, in this case- "Testing_data_of_ten_patients"
  df=pd.read_csv("/content/drive/MyDrive/FYP/first/final/Data/Omnyk_data/Testing_data_of_ten_patients/{}".format(i))
  df_new= df[['IBI']]
  df=df_new[(df_new["IBI"]>=400) & (df_new["IBI"]<=1333)]
  raw_nn=np.array(df).squeeze()
  t = 5
  t = t*60
  sums = 0
  for idx in range(len(raw_nn)):
    sums += raw_nn[idx]*0.001
    if (sums >= t):
      break

  nn_intervals = raw_nn[0:idx]
  res = time_domain_pm(nn_intervals)
  testDict = {"MEAN_RR":[res[0]], "MEDIAN_RR":[res[1]], "SDRR":[res[2]], "RMSSD":[res[3]], "SDSD":[res[4]], "HR":[res[5]], "pNN25":[res[6]], "pNN50":[res[7]]}
  testFile = pd.DataFrame(testDict)

  for j in range(idx+1, len(raw_nn)):
    nn_intervals = raw_nn[j-idx:j]
    res = time_domain_pm(nn_intervals)
    testDict = {"MEAN_RR":res[0], "MEDIAN_RR":res[1], "SDRR":res[2], "RMSSD":res[3], "SDSD":res[4], "HR":res[5], "pNN25":res[6], "pNN50":res[7]}
    testFile = testFile.append(testDict, ignore_index=True)
  count = 1
  test_sample=testFile


  #calculating parameters for each patient

  test_sample.to_csv("{}_patient_parameters.csv".format(i.split('.')[0]))

  count = count+1